# **UC4. Is there something wrong with my network subscribers? DL Approach**


In our network, the Provider Edge devices also function as Broadband Network Gateways (BNGs), providing Internet access to subscribers based on their region

The challenge is to determine whether the number of subscribers in a BNG is normal or not. Since this metric changes over time due to seasonality, setting a fixed threshold is not feasible. Instead, a mechanism is needed to learn the normal values and detect deviations as anomalies.

The goal is not to optimize these algorithms but to demonstrate how they can be programmed, trained, and how they perform in this use case. The same methodology can be applied to other metrics of interest.

In [ ]:
pip install tensorflow==2.19.0


# Extra libraries to install

# Connection to drive and path definition (Just for Google Colab Lab)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
path_files =('/content/drive/MyDrive/Colab Notebooks/Files')

# ***OR***

# Connection path definition (Just for AWS Jupiter Notebook)

In [ ]:
import sys
path_files ='./Files'

# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error
import time
from calendar import timegm, monthrange
from datetime import datetime, timedelta
import math
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
from tensorflow import keras
from os import listdir
from os.path import isfile, join
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, Dropout, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import CategoricalAccuracy,Accuracy,BinaryAccuracy

# [1] Data Collection

 We will load a data set that contains number of subscribers for PE-4 for a period of 30 days, with **5 minute frequency**

In [ ]:
metric_df=pd.read_csv(join(path_files,'bng_subscribers_metric.csv'),index_col=0)

In [ ]:
metric_df

In [ ]:
ds_column = []
for i,obs in metric_df.iterrows():
    #new_timestamp = datetime.fromtimestamp(obs['timestamp']).strftime('%Y-%m-%d %H:%M:%S')
    new_timestamp = datetime.fromtimestamp(obs['timestamp'])
    ds_column.append(new_timestamp)
metric_df['ds']=ds_column
metric_df['y']=metric_df['bng_subscribers']



In [ ]:
metric_df

This is how the data looks like for the target metric:

In [ ]:
# Visualize data using seaborn
sns.set(rc={'figure.figsize':(12,8)})
sns.lineplot(x=metric_df['ds'][:2000], y=metric_df['bng_subscribers'][:2000])
plt.title('BNG Subscribers')

# LSTM based autoencoder (RNN)

**Approach 4:** We will start exploring the possibility to detect anomalies using Neural Networks and deep learning. In particular in this case, we will start with using a sequence based autoencoder. The sequence based aspect results in the approach of using LSTM. LSTM is well built for sequences, so the intuition says that it may be a good choice. However, it has been reported that LSTM does not work specially well with auto-regresive signals, so we will need to check how well in performs in our case.

We will use 95% for training and 5% for testing or validation. The reason we use a larger percentage for training is that, in this particular test, we do not have many observations, only around 8,600.

Data Nomalization:

In [ ]:
scaler = StandardScaler()
scaler = scaler.fit(metric_df[['y']])
metric_df['y_scaled']=scaler.transform(metric_df[['y']])


# [3] DataSet Splitting

In [ ]:
train_pct = 0.95
train_size = int(metric_df.shape[0]*train_pct)
test_size = metric_df.shape[0]-train_size

# [4] Formatting for LSTM

In [ ]:
time_steps = 72


Create sequences of Data:

In [ ]:
Xs, ys = [], []
for i in range(metric_df.shape[0] - time_steps):
    v = metric_df[['y_scaled']].iloc[i:(i + time_steps)].values
    Xs.append(v)
    ys.append(metric_df.y_scaled.iloc[i + time_steps-1])
x = np.array(Xs)
y = np.array(ys)

In [ ]:
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# [5] Model Definition

In [ ]:
model = Sequential()
model.add(LSTM(units=128,input_shape=(x_train.shape[1],x_train.shape[2]),return_sequences=False))
model.add(Dropout(rate=0.2))
model.add(RepeatVector(n=x_train.shape[1]))
model.add(LSTM(units=128,return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(TimeDistributed(keras.layers.Dense(units=x_train.shape[2])))
model.add(Dropout(rate=0.2))

model.compile(loss='mse',optimizer =Adam(),metrics=['mse'])

The model compilation command defines how training will be conducted. The "compile" method specifies the loss function (MSE in this case), which depends on the use case. The optimizer (here, "adam") controls weight updates during training. Additionally, selected metrics help monitor and guide the training process.

In [ ]:
model.summary()

This command provides output shape information for each layer, useful for troubleshooting. It also shows the number and distribution of model parameters, affecting training complexity. In this case, most parameters are concentrated in the second LSTM layer of the decoder. Typically, they are found in the last dense layers, but in our simple example, their contribution to complexity is minimal.

In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True,show_dtype=True,show_layer_names=True,show_layer_activations=True)

# [6] Model Training

In [ ]:
filepath=join(path_files,"weights_autoencoder_auto_mse_best.weights.h5")
checkpoint = ModelCheckpoint(filepath, monitor='mse', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

A **checkpoint** ensures model weights are saved during training to prevent data loss if the process is interrupted (e.g., crashes, disconnections). It defines:  
- **Metric to monitor** (MSE).  
- **When to save weights** (**save_best_only**).  
- **Condition to save** (if the metric improves).  
- **File to store weights**.  

This allows resuming training with the last best weights, maintaining progress even if the process is interrupted.

**Model Training:** This command initiates the training process. The first two parameters are typically the training data (X, y), but for an autoencoder, we use **X, X** so the label is the same as the input data.  

An **epoch** is one full iteration over the dataset, and the number of epochs needed depends on the use case. You can set **callbacks** to stop training if the model's performance doesn't improve by a certain percentage. In this case, we’ve set 50 epochs.



---



In [ ]:
history = model.fit(x_train,x_train, epochs =50 , batch_size=128,validation_split=0.1,shuffle = False,callbacks=callbacks_list,verbose=1)

In [ ]:
model.save_weights(filepath)

In [ ]:
model.save(join(path_files,'anomaly_model_autoencoder_best.keras'))

# [7] Model Evaluation

In [ ]:
y_test_pred = model.predict(x_test)

In [ ]:
score, acc = model.evaluate(x_test, x_test)

In [ ]:
print('Test score:', score)
print('Test accuracy:', acc)

## [8] Anomaly Detection / Inference

In [ ]:
y_pred = model.predict(x)
y_pred = y_pred[:, -1, 0].reshape(-1, 1)
y = y.reshape(-1, 1)
mae_loss = np.abs(y_pred - y)
mae_threshold = np.percentile(mae_loss, 99)

score_df = pd.DataFrame(index=metric_df[time_steps:].ds)
score_df['loss'] = mae_loss
score_df['threshold'] = mae_threshold
score_df['anomaly'] = score_df.loss > score_df.threshold
score_df['y'] = y
score_df['y_pred'] = y_pred

total_anomalies = score_df[score_df.anomaly == True]

# --- Plot global anomalies ---

In [ ]:
sns.lineplot(x= score_df.index, y= score_df.loss,label='loss')
sns.lineplot(x = score_df.index, y =score_df.threshold,label='threshold')

In [ ]:
scope = 2000
scope_time = score_df.index[scope]
anomaly_scope = total_anomalies.index <= scope_time
sns.lineplot(
      x= score_df[:scope].index,
      y= scaler.inverse_transform(score_df[:scope].y.values.reshape(1,-1)).reshape(-1),
      label='actual bng_subscribers'
    )

sns.lineplot(
      x= score_df[:scope].index,
      y= scaler.inverse_transform(score_df[:scope].y_pred.values.reshape(1,-1)).reshape(-1),
      label='predicted bng_subscribers'
    )

sns.scatterplot(
      x= total_anomalies[anomaly_scope].index,
      y= scaler.inverse_transform(total_anomalies[anomaly_scope].y.values.reshape(1,-1)).reshape(-1),
      color=sns.color_palette()[3],
      s=52,
      label='anomaly'
    )

# Feed Forward Dense layers based Autoencoder (FNN)


Create sequences of size 'time_steps' for the input and the label

# [3] Input Formatting

In [ ]:
time_steps = 2016


In [ ]:
Xs, ys = [], []
for i in range(metric_df.shape[0] - time_steps):
    v = metric_df[['y_scaled']].iloc[i:(i + time_steps)].values
    Xs.append(v)
    ys.append(metric_df.y_scaled.iloc[i + time_steps-1])
x = np.array(Xs)
y = np.array(ys)

# [4] DataSet Splitting

In [ ]:
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# [5] Model Definition

In [ ]:
model = keras.Sequential()

model.add(keras.layers.Dense(units=1024,input_shape=(x_train.shape[1],),activation='tanh'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=512,activation='tanh'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=128,activation='tanh'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=512,activation='tanh'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=x_train.shape[1]))
model.add(keras.layers.Dropout(rate=0.2))



Model compilation:

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0005)

model.compile(loss='mse',optimizer =opt,metrics=['mse'])

Model brief:

In [ ]:
model.summary()

A **checkpoint** ensures model weights are saved during training to prevent data loss if the process is interrupted (e.g., crashes, disconnections). It defines:  
- **Metric to monitor** (MSE).  
- **When to save weights** (**save_best_only**).  
- **Condition to save** (if the metric improves).  
- **File to store weights**.  

This allows resuming training with the last best weights, maintaining progress even if the process is interrupted.

In [ ]:
filepath=join(path_files,"weights_autoencoder_dense_mse_best.weights.h5")
checkpoint = ModelCheckpoint(filepath, monitor='mse', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

**Model Training:** This command initiates the training process. The first two parameters are typically the training data (X, y), but for an autoencoder, we use **X, X** so the label is the same as the input data.  

An **epoch** is one full iteration over the dataset, and the number of epochs needed depends on the use case. You can set **callbacks** to stop training if the model's performance doesn't improve by a certain percentage. In this case, we’ve set 50 epochs.

# [6] Model Training





In [ ]:
history = model.fit(x_train,x_train, epochs =50 , batch_size=128,validation_split=0.1,shuffle = False,callbacks=callbacks_list,verbose=1)

In [ ]:
model.save_weights(filepath)

# [7] Model Evaluation



In [ ]:
y_pred = model.predict(x)

In [ ]:
y_pred = model.predict(x)
y_pred = y_pred[:, -1]  # Último valor reconstruido de cada secuencia
y_pred = y_pred.reshape(-1, 1)
y = y.reshape(-1, 1)

# Calcular error absoluto (MAE)

In [ ]:
mae_loss = np.abs(y_pred - y)


# Definir umbral para anomalías (percentil 99)

In [ ]:
mae_threshold = np.percentile(mae_loss, 99)

# Crear DataFrame con resultados

In [ ]:
score_df = pd.DataFrame(index=metric_df[(time_steps):].ds)
score_df['loss'] = mae_loss
score_df['threshold'] = mae_threshold
score_df['anomaly'] = score_df.loss > score_df.threshold
score_df['y'] = y
score_df['y_pred']=y_pred


# Filtrar anomalías

In [ ]:

total_anomalies = score_df[score_df.anomaly == True]
total_anomalies.head()

In [ ]:
sns.lineplot(x= score_df[:scope].index, y= score_df[:scope].loss,label='loss')
sns.lineplot(x = score_df[:scope].index, y =score_df[:scope].threshold,label='threshold')

In the previous visualization, we are capturing the "difference" between the predicted value and the actual value of the metric. Effectively, when the loss is high, it means the metric is not behaving as the model has "learned," so it can be considered an anomaly. We have calculated a threshold as the 99th percentile of the loss values, so we consider that, beyond this point, a value will be determined as anomalous.

# --- Visualización de resultados ---

In [ ]:
scope = 2000
scope_time = score_df.index[scope]
anomaly_scope = total_anomalies.index <= scope_time
sns.lineplot(
      x= score_df[:scope].index,
      y= scaler.inverse_transform(score_df[:scope].y.values.reshape(1,-1)).reshape(-1),
      label='actual bng_subscribers'
    )

sns.lineplot(
      x= score_df[:scope].index,
      y= scaler.inverse_transform(score_df[:scope].y_pred.values.reshape(1,-1)).reshape(-1),
      label='predicted bng_subscribers'
    )

sns.scatterplot(
      x= total_anomalies[anomaly_scope].index,
      y= scaler.inverse_transform(total_anomalies[anomaly_scope].y.values.reshape(1,-1)).reshape(-1),
      color=sns.color_palette()[3],
      s=52,
      label='anomaly'
    )